Imports

Load log files

In [10]:
%load_ext autoreload
%autoreload 2
from data_prep_helper import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. Read in log files

In [11]:
empatica_data_a3 = get_empatica_data(a3=True)
empatica_data_a4 = get_empatica_data(a3=False)
keylog_data_a3 = get_keylog_data(a3=True)
keylog_data_a4 = get_keylog_data(a3=False)
_, keylog_data_a3 = clip_for_start_end_times(empatica_data_a3, keylog_data_a3, a3=True)
_, keylog_data_a4 = clip_for_start_end_times(empatica_data_a4, keylog_data_a4, a3=False)

# start times from 0
for df in keylog_data_a3.values():
    df["T_s"] = df["T_s"] - df["T_s"].min()
    df['Time'] = pd.to_datetime(df['Time'])
    df = df.sort_values('Time').reset_index(drop=True)
    df['change_length'] = df['range_length'] + 1
for df in keylog_data_a4.values():
    df["T_s"] = df["T_s"] - df["T_s"].min()
    df['Time'] = pd.to_datetime(df['Time'])
    df = df.sort_values('Time').reset_index(drop=True)
    df['change_length'] = df['range_length'] + 1


Remove participants without start data

In [12]:
for p in a3_participants_to_remove_keylogs:
    keylog_data_a3.pop(p)
for p in a4_participants_to_remove_keylogs:
    keylog_data_a4.pop(p)
print(len(keylog_data_a3), len(keylog_data_a4))

13 14


Check data

In [13]:
pd.set_option('display.max_rows', 200)

In [14]:
keylog_data_a3['P11'].head(30)

,file_name,Time,T_s,start_line,start_character,end_line,end_character,range_offset,range_length,text,Time_s
0,inference.py,2024-03-13 01:22:30.300,0.000,16,0,16,0,401,0,S,0.000
1,inference.py,2024-03-13 01:22:30.498,0.198,16,1,16,1,402,0,t,0.198
2,inference.py,2024-03-13 01:22:30.656,0.356,16,2,16,2,403,0,a,0.356
3,inference.py,2024-03-13 01:22:31.321,1.021,16,3,16,3,404,0,r,1.021
4,inference.py,2024-03-13 01:22:31.551,1.251,16,4,16,4,405,0,t,1.251
5,inference.py,2024-03-13 01:22:31.889,1.589,16,5,16,5,406,0,i,1.589
6,inference.py,2024-03-13 01:22:32.132,1.832,16,6,16,6,407,0,n,1.832
7,inference.py,2024-03-13 01:22:32.603,2.303,16,7,16,7,408,0,g,2.303
8,inference.py,2024-03-13 01:22:38.235,7.935,16,0,16,8,401,8,,7.935
9,inference.py,2024-03-13 01:42:48.070,1217.770,71,0,71,0,1917,0,d,1217.770


- Line no's start from 0
- characters end at 200


Check start times are okay

In [17]:
# for every df print the largest difference between two consecutive timestamps and the index of this difference
print("A3:")
for p, df in keylog_data_a3.items():
    max_diff = df['Time_s'].diff().max()
    max_diff_idx = df['Time_s'].diff().idxmax()
    max_diff_time = df['Time_s'][max_diff_idx]
    max_diff_time_prev = df['Time_s'][max_diff_idx - 1]
    print(f"{p}, Max time difference  between keystrokes: {round(max_diff/60,1)}m, at index: {max_diff_idx},  starting: {round(max_diff_time_prev)}, ending:{round(max_diff_time)}")

print("\nA4:")
for p, df in keylog_data_a4.items():
    max_diff = df['Time_s'].diff().max()
    max_diff_idx = df['Time_s'].diff().idxmax()
    max_diff_time = df['Time_s'][max_diff_idx]
    max_diff_time_prev = df['Time_s'][max_diff_idx - 1]
    print(f"{p}, Max time difference  between keystrokes: {round(max_diff/60,1)}m, at index: {max_diff_idx},  starting: {round(max_diff_time_prev)}, ending:{round(max_diff_time)}")


A3:
P5, Max time difference  between keystrokes: 30.5m, at index: 6898,  starting: 9853, ending:11683
P11, Max time difference  between keystrokes: 43.1m, at index: 70,  starting: 2013, ending:4601
P12, Max time difference  between keystrokes: 8.2m, at index: 469,  starting: 2118, ending:2609
P13, Max time difference  between keystrokes: 14.9m, at index: 292,  starting: 1838, ending:2733
P19, Max time difference  between keystrokes: 6.8m, at index: 2552,  starting: 4866, ending:5273
P20, Max time difference  between keystrokes: 4.8m, at index: 1606,  starting: 4156, ending:4446
P21, Max time difference  between keystrokes: 33.2m, at index: 60,  starting: 766, ending:2759
P24, Max time difference  between keystrokes: 10.8m, at index: 8,  starting: 1, ending:651
P28, Max time difference  between keystrokes: 25.4m, at index: 708,  starting: 3558, ending:5084
P29, Max time difference  between keystrokes: 6.2m, at index: 2131,  starting: 5114, ending:5488
P30, Max time difference  between k

Function for applying changes

In [18]:
def apply_change(file_lines, start_line, start_char, end_line, end_char, new_text):
    """
    Applies a change to the file_lines list.

    Parameters:
    - file_lines: List of strings representing the file content.
    - start_line: 0-based starting line number.
    - start_char: 0-based starting character index in start_line.
    - end_line: 0-based ending line number.
    - end_char: 0-based ending character index in end_line.
    - new_text: The text to replace the specified range with.

    Returns:
    - Updated list of file lines.
    """
    # Use the indices directly since they are 0-based
    start_idx = start_line
    end_idx = end_line

    # If the change is beyond the current file length, extend file_lines
    if end_idx >= len(file_lines):
        # Calculate how many lines need to be added
        lines_to_add = end_idx - len(file_lines) + 1
        file_lines.extend(['\n'] * lines_to_add)

    # Ensure start_idx and end_idx are within bounds
    start_idx = min(start_idx, len(file_lines) - 1)
    end_idx = min(end_idx, len(file_lines) - 1)

    # Handle single-line changes
    if start_idx == end_idx:
        line = file_lines[start_idx]
        line_length = len(line)
        start_char = min(start_char, line_length)
        end_char = min(end_char, line_length)
        new_line = line[:start_char] + new_text + line[end_char:]
        file_lines[start_idx] = new_line
    else:
        # Handle multi-line changes
        first_line = file_lines[start_idx]
        last_line = file_lines[end_idx]

        # Ensure start_char and end_char are within line lengths
        start_char = min(start_char, len(first_line))
        end_char = min(end_char, len(last_line))

        # Extract parts before start_char and after end_char
        before = first_line[:start_char]
        after = last_line[end_char:]

        # Replace the range with new_text
        new_text_lines = new_text.split('\n')

        # Update the first line
        file_lines[start_idx] = before + new_text_lines[0] + '\n'

        # Remove the lines that are being replaced
        del file_lines[start_idx + 1:end_idx + 1]

        # Insert any additional lines from new_text
        for i, line in enumerate(new_text_lines[1:]):
            file_lines.insert(start_idx + 1 + i, line + '\n')

        # Append the 'after' part to the last line of new_text
        if after.strip():
            last_new_line_idx = start_idx + len(new_text_lines) - 1
            file_lines[last_new_line_idx] = file_lines[last_new_line_idx].rstrip('\n') + after

    return file_lines

TODO: 
- FIX so that removals also work
- also save start version of the files

Function for generating snapshots using apply_changes above

In [21]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        return f.readlines()

# create directories
snapshots_dir_a3 = 'code_snapshots_a3'
snapshots_dir_a4 = 'code_snapshots_a4'
os.makedirs(snapshots_dir_a3, exist_ok=True)
os.makedirs(snapshots_dir_a4, exist_ok=True)


def create_snapshots(p, df, a3=True, snapshot_interval_minutes=10):
    # Define snapshot interval
    snapshot_interval = pd.Timedelta(minutes=snapshot_interval_minutes)
    start_time = df['Time'].min()
    next_snapshot_time = start_time + snapshot_interval
    snapshot_counter = 1

    code_file_path_bustersagent = f"/Users/feyzjan/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GatechCourses/CS 8903 Research/A3_DataFiles/{p}/Session_Start_{p}/tracking/busters.py"
    code_file_path_inference = f"/Users/feyzjan/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GatechCourses/CS 8903 Research/A3_DataFiles/{p}/Session_Start_{p}/tracking/inference.py"
    
    code_file_path_neuralnet = f"/Users/feyzjan/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GatechCourses/CS 8903 Research/A4_DataFiles/{p}/Session_Start_{p}/NeuralNet.py"
    code_file_path_neuralnetutil = f"/Users/feyzjan/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GatechCourses/CS 8903 Research/A4_DataFiles/{p}/Session_Start_{p}/NeuralNetUtil.py"
    assignment = "a3" if a3 else "a4"
    snapshots_dir = snapshots_dir_a3 if a3 else snapshots_dir_a4
    try:
        if a3 == True:
            busters_lines = read_file(code_file_path_bustersagent)
            inference_lines = read_file(code_file_path_inference)
            files_content = {
                'bustersAgents.py': busters_lines,
                'inference.py': inference_lines
            }
        else:

            neuralnet_lines = read_file(code_file_path_neuralnet)
            neuralnetutil_lines = read_file(code_file_path_neuralnetutil)
            files_content = {
                'NeuralNet.py': neuralnet_lines,
                'NeuralNetUtil.py': neuralnetutil_lines
            }
    except FileNotFoundError:
        print(f"Warning: Files not found for participant {p}.")
        
    # print(p, len(busters_lines), len(inference_lines))
    # print(p, len(neuralnet_lines), len(neuralnetutil_lines))


    # Iterate through each change
    for index, row in df.iterrows():
        change_time = row['Time']
        
        # Check if it's time to take a snapshot
        while change_time >= next_snapshot_time:
            # Save the current state of the files
            snapshot_filename = f'{assignment}_{p}_{snapshot_counter}'
            snapshot_path = os.path.join(snapshots_dir, snapshot_filename)

            # Uncomment after testing
            os.makedirs(snapshot_path, exist_ok=True)
            
            for file_name, lines in files_content.items():
                with open(os.path.join(snapshot_path, file_name), 'w') as f:
                    f.writelines(lines)
            
            print(f"Saved snapshot: {snapshot_filename}")
            
            # Update for next snapshot
            snapshot_counter += 1
            next_snapshot_time += snapshot_interval
        
        # Apply the change
        file_name = row['file_name']
        start_line = row['start_line']
        start_character = row['start_character']
        end_line = row['end_line']
        end_character = row['end_character']
        new_text = row['text']
        
        if file_name not in files_content:
            print(f"Warning: File {file_name} not recognized.")
            continue
        
        # Apply the change
        files_content[file_name] = apply_change(
            files_content[file_name],
            start_line,
            start_character,
            end_line,
            end_character,
            new_text
        )

In [22]:
snapshot_interval_minutes = 5
for p in keylog_data_a3.keys():
    create_snapshots(p, keylog_data_a3[p], a3=True, snapshot_interval_minutes=snapshot_interval_minutes)

for p in keylog_data_a4.keys():
    create_snapshots(p, keylog_data_a4[p], a3=False, snapshot_interval_minutes=snapshot_interval_minutes)



Saved snapshot: a3_P5_1
Saved snapshot: a3_P5_2
Saved snapshot: a3_P5_3
Saved snapshot: a3_P5_4
Saved snapshot: a3_P5_5
Saved snapshot: a3_P5_6
Saved snapshot: a3_P5_7
Saved snapshot: a3_P5_8
Saved snapshot: a3_P5_9
Saved snapshot: a3_P5_10
Saved snapshot: a3_P5_11
Saved snapshot: a3_P5_12
Saved snapshot: a3_P5_13
Saved snapshot: a3_P5_14
Saved snapshot: a3_P5_15
Saved snapshot: a3_P5_16
Saved snapshot: a3_P5_17
Saved snapshot: a3_P5_18
Saved snapshot: a3_P5_19
Saved snapshot: a3_P5_20
Saved snapshot: a3_P5_21
Saved snapshot: a3_P5_22
Saved snapshot: a3_P5_23
Saved snapshot: a3_P5_24
Saved snapshot: a3_P5_25
Saved snapshot: a3_P5_26
Saved snapshot: a3_P5_27
Saved snapshot: a3_P5_28
Saved snapshot: a3_P5_29
Saved snapshot: a3_P5_30
Saved snapshot: a3_P5_31
Saved snapshot: a3_P5_32
Saved snapshot: a3_P5_33
Saved snapshot: a3_P5_34
Saved snapshot: a3_P5_35
Saved snapshot: a3_P5_36
Saved snapshot: a3_P5_37
Saved snapshot: a3_P5_38
Saved snapshot: a3_P5_39
Saved snapshot: a3_P5_40
Saved sna

### Copy over files from a3_files_to_copy/a4_files_to_copy

In [23]:
import os
import shutil

def move_assignment_files(a3=True):
    # Define the directories
    if a3:
        code_snapshots_dir = 'code_snapshots_a3'
        files_to_copy_dir = 'a3_files_to_copy'
    else:
        files_to_copy_dir = 'a4_files_to_copy'
        code_snapshots_dir = 'code_snapshots_a4'

    # Check if the directories exist
    if not os.path.isdir(code_snapshots_dir):
        print(f"Error: Directory '{code_snapshots_dir}' does not exist.")
        return

    if not os.path.isdir(files_to_copy_dir):
        print(f"Error: Directory '{files_to_copy_dir}' does not exist.")
        return

    # Get a list of snapshot directories
    snapshot_dirs = [os.path.join(code_snapshots_dir, d) for d in os.listdir(code_snapshots_dir) if os.path.isdir(os.path.join(code_snapshots_dir, d))]

    # Process each snapshot directory
    for snapshot_dir in snapshot_dirs:
        print(f"Processing '{snapshot_dir}'...")

        # Step 1: Copy contents of a3_files_to_copy into the snapshot directory
        copy_files(files_to_copy_dir, snapshot_dir)

        # Step 2: Move 'inference.py' and 'bustersAgents.py' into the 'tracking' folder
        if a3:
            target_dir = os.path.join(snapshot_dir, 'tracking')
        else:
            target_dir = snapshot_dir

        # Ensure the 'tracking' directory exists
        if not os.path.isdir(target_dir):
            print(f"Error: Tracking directory '{target_dir}' does not exist after copying.")
            continue

        if a3:
            move_file_if_exists(os.path.join(snapshot_dir, 'inference.py'), target_dir)
            move_file_if_exists(os.path.join(snapshot_dir, 'bustersAgents.py'), target_dir)
        else:
            move_file_if_exists(os.path.join(snapshot_dir, 'NeuralNet.py'), target_dir)
            move_file_if_exists(os.path.join(snapshot_dir, 'NeuralNetUtil.py'), target_dir)

    print("All snapshots have been processed.")

def copy_files(src_dir, dest_dir):
    """
    Copy all files and directories from src_dir to dest_dir.
    """
    for item in os.listdir(src_dir):
        s = os.path.join(src_dir, item)
        d = os.path.join(dest_dir, item)
        try:
            if os.path.isdir(s):
                if os.path.exists(d):
                    shutil.rmtree(d)
                shutil.copytree(s, d)
            else:
                shutil.copy2(s, d)
        except Exception as e:
            print(f"Error copying '{s}' to '{d}': {e}")

def move_file_if_exists(file_path, dest_dir):
    """
    Move file to dest_dir if it exists.
    """
    if os.path.isfile(file_path):
        try:
            shutil.move(file_path, dest_dir)
            print(f"Moved '{os.path.basename(file_path)}' to '{dest_dir}'.")
        except Exception as e:
            print(f"Error moving '{file_path}' to '{dest_dir}': {e}")
    else:
        print(f"Warning: '{file_path}' does not exist and cannot be moved.")

In [25]:
# move_assignment_files(True)
# move_assignment_files(False)

Processing 'code_snapshots_a3/a3_P43_2_31'...
Moved 'inference.py' to 'code_snapshots_a3/a3_P43_2_31/tracking'.
Moved 'bustersAgents.py' to 'code_snapshots_a3/a3_P43_2_31/tracking'.
Processing 'code_snapshots_a3/a3_P43_1_16'...
Moved 'inference.py' to 'code_snapshots_a3/a3_P43_1_16/tracking'.
Moved 'bustersAgents.py' to 'code_snapshots_a3/a3_P43_1_16/tracking'.
Processing 'code_snapshots_a3/a3_P34_1_3'...
Moved 'inference.py' to 'code_snapshots_a3/a3_P34_1_3/tracking'.
Moved 'bustersAgents.py' to 'code_snapshots_a3/a3_P34_1_3/tracking'.
Processing 'code_snapshots_a3/a3_P34_1_4'...
Moved 'inference.py' to 'code_snapshots_a3/a3_P34_1_4/tracking'.
Moved 'bustersAgents.py' to 'code_snapshots_a3/a3_P34_1_4/tracking'.
Processing 'code_snapshots_a3/a3_P43_1_11'...
Moved 'inference.py' to 'code_snapshots_a3/a3_P43_1_11/tracking'.
Moved 'bustersAgents.py' to 'code_snapshots_a3/a3_P43_1_11/tracking'.
Processing 'code_snapshots_a3/a3_P5_33'...
Moved 'inference.py' to 'code_snapshots_a3/a3_P5_33/t

Run the autograder
- TODO: 
    -  Install pypy
    -  I



In [28]:
import os
import subprocess


def run_autograder(a3):
    error_count = 0
    success_count = 0
    if a3:
        code_snapshots_dir = 'code_snapshots_a3'
    else:
        code_snapshots_dir = 'code_snapshots_a4'

    # Get a list of snapshot directories
    snapshot_dirs = [
        os.path.join(code_snapshots_dir, d)
        for d in os.listdir(code_snapshots_dir)
        if os.path.isdir(os.path.join(code_snapshots_dir, d))
    ]

    # Process each snapshot directory
    for snapshot_dir in snapshot_dirs:
        print(f"\nProcessing '{snapshot_dir}'...")

        # Run the command
        if a3:
            cmd = ['python', 'autograder.py', '-q', 'q1', '--no-graphics']
            dir = os.path.join(snapshot_dir, 'tracking')
        else:
            cmd = ['python', 'autograder.py']
            # cmd = ['pypy', 'autograder.py']
            dir = snapshot_dir
        result = subprocess.run(cmd, cwd=dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Print the output
        # print("Standard Output:")
        # print(result.stdout)
        if result.stderr:
            print("Standard Error:")
            error_count += 1
            # print(result.stderr)
            pass
        else:
            success_count += 1
            print("No errors found.")
      
    print("\nAll snapshots have been processed.")
    print(f"Success count: {success_count}")
    print(f"Error count: {error_count}")


In [30]:
run_autograder(a3=True)
# run_autograder(a3=False)


Processing 'code_snapshots_a3/a3_P43_2_31'...
Standard Error:

Processing 'code_snapshots_a3/a3_P43_1_16'...
Standard Error:

Processing 'code_snapshots_a3/a3_P34_1_3'...
Standard Error:

Processing 'code_snapshots_a3/a3_P34_1_4'...
Standard Error:

Processing 'code_snapshots_a3/a3_P43_1_11'...
Standard Error:

Processing 'code_snapshots_a3/a3_P5_33'...
Standard Error:

Processing 'code_snapshots_a3/a3_P43_1_18'...
Standard Error:

Processing 'code_snapshots_a3/a3_P43_1_20'...
Standard Error:

Processing 'code_snapshots_a3/a3_P5_34'...
Standard Error:

Processing 'code_snapshots_a3/a3_P43_2_3'...
Standard Error:

Processing 'code_snapshots_a3/a3_P43_2_4'...
Standard Error:

Processing 'code_snapshots_a3/a3_P13_14'...
Standard Error:

Processing 'code_snapshots_a3/a3_P5_51'...
Standard Error:

Processing 'code_snapshots_a3/a3_P42_14'...
Standard Error:

Processing 'code_snapshots_a3/a3_P12_3'...
Standard Error:

Processing 'code_snapshots_a3/a3_P42_13'...
Standard Error:

Processing 'c